**IMPORTING LIBRARIES**

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

**READING EXCEL AND DROP DATE COLUMN**

In [39]:
data=pd.read_excel('data_akbilgic.xlsx',header=1)
data=data.loc[0:529] #keep the rows from 0 to 529 (first 530 rows)
data=data.drop(columns=['date']) #drop date column date

**GRAPHICAL VIEW OF OUR DATA**

In [ ]:
data.reset_index(inplace=True)#add index

plt.plot(data['ISE'])
plt.savefig('./ise_plot.png')
plt.show()




**COLLECT DATA ONLY FROM ISE**

In [ ]:
df = data['ISE'].to_frame(name='ISE')

**SCALING**

In [41]:
df_1 = df.copy() #MAKE A COPY
scaler = MinMaxScaler(feature_range=(0, 1)) #SCALING OUR DATA
df = scaler.fit_transform(np.reshape(df['ISE'].values, (df.shape[0], 1)))
df = pd.DataFrame(data=df, columns=['ISE'])


**MAKE DATA SUPERVISED**

In [ ]:
def timeseries_to_supervised(df, n_in, n_out):
   agg = pd.DataFrame()

   for i in range(n_in, 0, -1):
      df_shifted = df.shift(i).copy()
      df_shifted.rename(columns=lambda x: ('%s(t-%d)' % (x, i)), inplace=True)
      agg = pd.concat([agg, df_shifted], axis=1)

   for i in range(0, n_out):
      df_shifted = df.shift(-i).copy()
      if i == 0:
         df_shifted.rename(columns=lambda x: ('%s(t)' % (x)), inplace=True)
      else:
         df_shifted.rename(columns=lambda x: ('%s(t+%d)' % (x, i)), inplace=True)
      agg = pd.concat([agg, df_shifted], axis=1)
   agg.dropna(inplace=True)
   return agg
n_in = 4 #Timeseries consists of 4 input steps.
n_out = 1
sdf = timeseries_to_supervised(df, n_in, n_out)
sdf

In [21]:
X, y = sdf[[('ISE(t-%d)' % i) for i in range(4, 0, -1)]].values, sdf['ISE(t)'].values

**PREPARING DATA FOR TESTING**

In [ ]:
len_data = X.shape[0]
print("Data size: %d" % len_data)
train_size = int(len_data * .5)# 50% for training and 50% for testing
print ("Train  data size: %d" % train_size)
print ("Test data size: %d" % (len_data - train_size))

xtr, ytr = X[:train_size, :], y[:train_size]
xte, yte = X[train_size:, :], y[train_size:]
print(xtr.shape, ytr.shape)
print(xte.shape, yte.shape)

**Create a model**

In [ ]:
batch_size = 1 #1
model = Sequential()

model.add(Dense(units=50, input_dim=xtr.shape[1], activation="relu"))#input dim is 4 as the timeseries steps.
model.add(Dense(50, activation="relu"))# linear activation functios was used as well
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

**TRAIN MODEL**

In [ ]:
model.fit(xtr,ytr, epochs=50, batch_size=batch_size, verbose=2)

In [25]:
trainPredict = model.predict(xtr, batch_size=batch_size)
testPredict = model.predict(xte, batch_size=batch_size)



In [26]:
ytr2d = np.reshape(ytr, (ytr.shape[0], 1))
yte2d = np.reshape(yte, (yte.shape[0], 1))

In [27]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(ytr2d)
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(yte2d)

In [28]:
print("Test MSE: ", mean_squared_error(testY, testPredict))
#print("Test MSE: ", sum(np.square(testY-testPredict))/testY.shape[0])#hand formula
print("Test MAE: ", sum(abs(testY-testPredict))/testY.shape[0])
print("Test R2: ", r2_score(testY, testPredict))
#print("Test R2: ", 1-(sum(np.square(testY-testPredict))/sum(np.square(testY-testY.mean()))))#hand formula

Test MSE:  0.014393961625893856
Test MAE:  [0.08833334]
Test R2:  -0.1429897290716613


**PLOT THE MODEL WITH TRAINING (LEFT) AND TESTING(RIGHT)**

In [ ]:
redicted = np.concatenate((trainPredict,testPredict),axis=0)

original = np.concatenate((trainY,testY),axis=0)
predicted = np.concatenate((trainPredict,testPredict),axis=0)
index = range(0, original.shape[0])
plt.plot(index,original, 'g')
plt.plot(index,predicted, 'r')
plt.axvline(df.index[train_size], c="b")
plt.savefig('./mlp_regression_results.png')
plt.show()

